In [6]:
import pandas as pd
from surprise import Dataset, Reader, SVD, KNNBasic
from surprise.model_selection import cross_validate

In [3]:
df = pd.read_parquet('dataset_ml.parquet')

In [4]:
df.head(5)

,userId,score,id,title
1,1,4.5,ns2186,latte and the magic waterstone
3,1,5.0,ns3663,"frankenstein’s monster’s monster, frankenstein"
4,1,5.0,as9500,kept woman
6,1,4.5,ns8282,the drowning
7,1,5.0,as6112,salome


In [8]:
reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(df[['userId','id','score']],reader=reader)

In [9]:
model = SVD()
cross_validate(model, data, measures=['RMSE','MAE'], cv=5, verbose=True)

In [ ]:
modelo_entrenado = data.build_full_trainset()

In [ ]:
modelo_entrenado

In [ ]:
model.fit(modelo_entrenado)

### Query: get_recomendation 
Consigna: recomendar películas a los usuarios basándose en películas similares, por lo que se debe encontrar la similitud de puntuación entre esa película y el resto de películas, se ordenarán según el score y devolverá una lista de Python con 5 valores, cada uno siendo el string del nombre de las películas con mayor puntaje, en orden descendente. Debe ser deployado como una función adicional de la API anterior y debe llamarse get_recommendation(titulo: str).

In [ ]:
def get_recomendation(title):
    prediccion = model.predict(title)
    if prediccion.est >= 3.5:
        print('s')
    respuesta = 0
    return {'Recomendación': respuesta}